In [1]:
import numpy as np
import cv2
import myutils
from imutils import contours

In [2]:
# 对轮廓排序函数
def sort_contours(cnts):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts] #用一个最小的矩形，把找到的形状包起来x,y,h,w
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][0]))
    return cnts

In [3]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

In [4]:
# 绘图展示
def cv_show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
# 读取一个模板图像
img = cv2.imread(r'D:\template-matching-ocr\images\ocr_a_reference.png')
cv_show('img', img)

In [6]:
# 灰度图
ref = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv_show('ref', ref)

In [7]:
# 二值图像
ref = cv2.threshold(ref, 10, 255, cv2.THRESH_BINARY_INV)[1]
cv_show('ref', ref)

In [8]:
# 计算轮廓
# cv2.findContours()函数接受的参数为二值图，cv2.RETR_EXTERNAL只检测外轮廓，cv2.CHAIN_APPROX_SIMPLE只保留终点坐标
# 返回的list中每个元素都是图像中的一个轮廓

refCnts, hierarchy = cv2.findContours(ref.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(img, refCnts, -1, (0,0,255),3)
cv_show('img', img)

In [9]:
# 排序
refCnts = sort_contours(refCnts)

In [10]:
digits = {}

In [11]:
# 遍历每一个轮廓
for (i, c) in enumerate(refCnts):
    #计算外接矩形并且resize成合适大小
    (x, y, w, h) = cv2.boundingRect(c)
    roi = ref[y:y+h, x:x+w]
    roi = cv2.resize(roi, (57, 88))
    
    # 每一个数字对应每一个模板
    digits[i] = roi

In [57]:
# 读取输入图像，预处理
image = cv2.imread(r'D:\template-matching-ocr\images\credit_card_01.png')
image = resize(image, width=300)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv_show('gray',gray)

In [58]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))

In [59]:
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

In [60]:
#礼帽操作，突出更明亮的区域
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel) 
cv_show('tophat',tophat) 
# 

In [61]:
sobelx = cv2.Sobel(tophat, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(tophat, cv2.CV_64F, 0, 1, ksize=3)
sobelx = cv2.convertScaleAbs(sobelx)
sobely = cv2.convertScaleAbs(sobely)
sobelxy = cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)
cv_show('sobelxy',sobelxy)

In [62]:
#通过闭操作（先膨胀，再腐蚀）将数字连在一起
sobelxy = cv2.morphologyEx(sobelxy, cv2.MORPH_CLOSE, rectKernel) 
cv_show('sobelxy',sobelxy)

In [63]:
# THRESH_OTSU会自动寻找合适的阈值，适合双峰，需把阈值参数设置为0
thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show('thresh',thresh)

In [64]:
# 再来一个闭操作
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
cv_show('thresh',thresh)

In [75]:
# 计算轮廓
threshCnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE )

cnts = threshCnts
cur_img = image.copy()
cv2.drawContours(cur_img, cnts, -1, (0,0,255),1)
cv_show('cur_img',cur_img)

In [77]:
# 遍历轮廓
locs = []
for (i, c) in enumerate(cnts):
    # 计算矩形
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    # 选择合适的区域，根据实际任务来
    if ar > 2.5 and ar < 4.0:
        if (w > 40 and w<55) and (h > 10 and h <20):
            locs.append((x, y, w, h))

# 将符合的轮廓从左到右排序
locs = sorted(locs, key=lambda x:x[0])

In [87]:
output = []

In [88]:
# 遍历每一个轮廓中的数字
for (i, (gX, gY, gW, gH)) in enumerate(locs):
    # 初始化数字列表
    groupOutput = []
    
    # 根据坐标提取每一个组
    group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
    cv_show('group',group)
    
    # 预处理
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    
    # 计算每一组的轮廓
    digitCnts, hierarchy = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_SIMPLE)
    digitCnts = sort_contours(digitCnts)
    
    # 计算每一组中的每一个数值
    for c in digitCnts:
        # 找到当前数值的轮廓，resize成合适的大小
        (x, y, w, h) = cv2.boundingRect(c)
        roi = group[y:y+h, x:x+w]
        roi = cv2.resize(roi, (57, 88))
        cv_show('roi',roi)
        
        # 计算匹配得分
        scores = []
        
        # 在模板中计算每一个得分
        for (digit, digtiROI) in digits.items():
            # 模板匹配
            result = cv2.matchTemplate(roi, digtiROI, cv2.TM_CCOEFF)
            (_, score, _, _) = cv2.minMaxLoc(result)
            scores.append(score)
            
        # 得到最合适的数字
        groupOutput.append(str(np.argmax(scores)))
    
    # 画出来
    cv2.rectangle(image, (gX - 5, gY - 5),
    (gX + gW + 5, gY + gH + 5), (0, 0, 255), 1)
    cv2.putText(image, "".join(groupOutput), (gX, gY - 15),
    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
    
    # 得到结果
    output.extend(groupOutput)

In [89]:
cv2.imshow("Image", image)
cv2.waitKey(0)

-1